In [ ]:
 !pip install -U diffusers
!pip install gradio

In [ ]:

from huggingface_hub import notebook_login
# Log in to your Hugging Face account
notebook_login()

# Token :  use your huggingface access token

In [ ]:
# prompt: write a gradio app for the above model impementation
import gradio as gr
from PIL import Image
import requests
import torch
from transformers import DetrImageProcessor, DetrForObjectDetection

In [ ]:
# you can specify the revision tag if you don't want the timm dependency
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50", revision="no_timm")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50", revision="no_timm")

In [ ]:
def inference(image):
    inputs = processor(images=image, return_tensors="pt")
    outputs = model(**inputs)

    # Convert outputs (bounding boxes and class logits) to COCO API
    # Let's only keep detections with score > 0.9
    target_sizes = torch.tensor([image.size[::-1]])
    results = processor.post_process_object_detection(
        outputs, target_sizes=target_sizes, threshold=0.9
    )[0]

    # Create a list of detected objects
    detected_objects = []
    for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
        box = [round(i, 2) for i in box.tolist()]
        detected_objects.append(
            f"Detected {model.config.id2label[label.item()]} with confidence "
            f"{round(score.item(), 3)} at location {box}"
        )

    return "\n".join(detected_objects)


In [ ]:
demo = gr.Interface(
   fn=inference,
   inputs=gr.Image(type="pil"),
   outputs=gr.Textbox(),
)


demo.launch()

In [ ]:




# prompt: rather than just printing the results i want the boxes to be drawn on the picture with labels to show what object it is


import torch
from diffusers import FluxPipeline
from huggingface_hub import notebook_login
from diffusers import StableDiffusionPipeline
import gradio as gr
from transformers import pipeline
from transformers import DetrImageProcessor, DetrForObjectDetection
from PIL import ImageDraw, ImageFont
from PIL import Image
import requests


# you can specify the revision tag if you don't want the timm dependency
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50", revision="no_timm")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50", revision="no_timm")


def inference(image):
 inputs = processor(images=image, return_tensors="pt")
 outputs = model(**inputs)


 # convert outputs (bounding boxes and class logits) to COCO API
 # let's only keep detections with score > 0.9
 target_sizes = torch.tensor([image.size[::-1]])
 results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]


 # Draw bounding boxes and labels on the image
 draw = ImageDraw.Draw(image)
 font = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSans-Regular.ttf", 15)  # You might need to adjust the font path


 for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
   box = [round(i, 2) for i in box.tolist()]
   draw.rectangle(box, outline="red", width=2)
   label_text = f"{model.config.id2label[label.item()]}"
   draw.text((box[0], box[1]), label_text, fill="red", font=font)


 return image


demo = gr.Interface(
   fn=inference,
   inputs=gr.Image(type="pil"),
   outputs=gr.Image(type="pil"),
)


demo.launch()
